---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [29]:
def date_sorter():
    import numpy as np
    from datetime import datetime

    set_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    set_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    set_a = pd.concat([set_1,set_2])
    set_a.reset_index(inplace=True)
    a_index = set_a['level_0']

    set_b = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    set_b.reset_index(inplace=True)
    b_index = set_b['level_0']

    set_c = df.str.extractall(r'((?:\d{1,2}))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    set_c.reset_index(inplace=True)
    c_index = set_c['level_0']

    set_d = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    set_d.reset_index(inplace=True)
    d_index = set_d['level_0']

    not_present_already = []
    for i in d_index:
        if(i not in a_index.values):
            not_present_already.append(i)
    not_present_already = np.asarray(not_present_already)
    set_d = set_d[set_d['level_0'].isin(not_present_already)]

    set_3 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    set_4 = df.str.extractall(r'^(\d{4})[^0-9]')
    set_e = pd.concat([set_3,set_4])
    set_e.reset_index(inplace=True)
    e_index = set_e['level_0']

    not_present_already = []
    for i in e_index:
        if not((i in b_index.values) | (i in c_index.values) | (i in d_index.values)):
            not_present_already.append(i)
    not_present_already = np.asarray(not_present_already)
    set_e = set_e[set_e['level_0'].isin(not_present_already)]

    set_a.columns = ['level_0','match','month','day','year']
    set_a['year'] = set_a['year'].apply(str)
    set_a['year'] = set_a['year'].apply(lambda x:'19'+x if len(x)<=2 else x)

    set_b[1] = set_b[1].str.replace(',','')
    set_b['day'] = set_b[1].apply(lambda x:x.split(' ')[0])
    set_b['year'] = set_b[1].apply(lambda x:x.split(' ')[1])
    set_b.columns = ['level_0','match','month','day-year','day','year']
    set_b = set_b.drop('day-year',axis=1)

    set_c.columns= ['level_0','match','day','month','year']
    set_c['day'] = set_c['day'].fillna(1)
    # set_c['day'] = set_c['day'].replace(np.nan,-1)
    # set_c['day'] = set_c['day'].apply(lambda x: 1 if (x==-1) else x)
    # set_c = set_c[set_c['day'].isnull() == False]
    set_c['month'] = set_c['month'].apply(lambda x: x[:3])
    set_c['month'] = pd.to_datetime(set_c.month,format='%b').dt.month

    set_d.columns = ['level_0','match','month','year']
    set_d['day'] = 1

    set_e.columns = ['level_0','match','year']
    set_e['month'] = 1
    set_e['day'] = 1

    res = pd.concat([set_a,set_b,set_c,set_d,set_e])
    res['date'] = pd.to_datetime(res['month'].apply(str) + '/' + res['day'].apply(str) + '/' + res['year'].apply(str))
    res = res.sort_values(by='level_0').set_index('level_0')

    finalResult = res['date']

    return pd.Series([i[0] for i in sorted(enumerate(finalResult), key=lambda x: x[1])], np.arange(500))
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      153
6      474
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     154
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    152
490    257
491    235
492    464
493    253
494    141
495    231
496    427
497    161
498    186
499    413
Length: 500, dtype: int64